In [161]:
import numpy as np 
import sklearn 
import DiffusionEMD
from DiffusionEMD import DiffusionCheb
import pandas as pd
import time

# Module

In [154]:
import scipy
import pygsp

class Graph_Fourier_MMD:
    def __init__(self, Graph = None):
        if Graph != None:
            self.G = Graph
            self.T = self.G.L.trace()
        else:
            raise ValueError("Graph Required")
            
    def feature_map(self, signals, method, filter_name):
        recip_filter = pygsp.filters.Filter(self.G, kernels = [lambda x : [i**(-1/2) if i != 0 else 0 for i in x]])
        heat_filter = pygsp.filters.Heat(self.G)

        if filter_name == 'heat':
            use_filter = heat_filter
        elif filter_name == "default":
            use_filter = recip_filter
        else:
            raise NameError("Filter name must either be 'heat' or 'default'")
        
        if method == "chebyshev":
            return use_filter.filter(signals, method = 'chebyshev') * (self.T)**(1/2)
        else:
            return use_filter.filter(signals, method = 'exact') * (self.T)**(1/2)

    def locality(self, signals, method='chebyshev', filter_name='default'):
        transformed = self.feature_map(signals, method, filter_name)
        return [np.linalg.norm(t) for t in transformed.T]

    def distance(self, signals, method='chebyshev', filter_name='default'):
        n = signals.shape[1]

        if n == 1:
            raise ValueError("Need more than two signals to compare")
            return False 

        transformed = self.feature_map(signals, method, filter_name)
        distances = scipy.spatial.pdist(signals.T)

        if n == 2:
            return distances[0,1]
        else:
            return distances

# Experiments on Minnesota Graph

## Generate Graph

In [174]:
G = Graph=pygsp.graphs.Minnesota()
GFMMD = Graph_Fourier_MMD(G)

## Generate Signals

Diffused at $\tau = 2^j$ for $j=0,4,8,12,16$

In [123]:
diracs = np.eye(G.N)
diffusions = pygsp.filters.Heat(G, tau = [2**0, 2**4, 2**8, 2**12, 2**16]).filter(diracs)

2022-08-24 12:05:17,337:[WARNING](pygsp.graphs.graph.lmax): The largest eigenvalue G.lmax is not available, we need to estimate it. Explicitly call G.estimate_lmax() or G.compute_fourier_basis() once beforehand to suppress the warning.


## Create Tables

One for time and one for overall locality measure

In [180]:
locality_measure = pd.DataFrame(columns = ['Method', 't=2^0', 't=2^4', 't=2^8', 't=2^12', 't=2^16' ])
runtime_table = pd.DataFrame(columns = ['Method', 'Compute Eigenvectors', 't=2^0', 't=2^4', 't=2^8', 't=2^12', 't=2^16' ])


Using Graph Fourier MMD with Chebyshev approximation of the filter $h(\lambda) = \lambda^{-1/2}\mathbf{1}\{\lambda \neq 0\}$:

In [181]:
row = {}
row['Method'] = 'Graph Fourier MMD (Chebyshev)'
time_row = row

for i in range(5):
    start_time = time.time()
    row[f't=2^{4*i}'] = str(np.mean(GFMMD.locality(diffusions[:,:,i])))
    runtime = time.time() - start_time
    time_row[f't=2^{4*i}'] = runtime

locality_measure.loc[0,:]=row

time_row['Compute Eigenvectors'] = 0
runtime_table.loc[0,:]=time_row

Using Graph Fourier MMD with exact filter $h(\lambda) = \lambda^{-1/2}\mathbf{1}\{\lambda \neq 0\}$:

In [182]:
G = Graph=pygsp.graphs.Minnesota()
GFMMD = Graph_Fourier_MMD(G)

row = {}
row['Method'] = 'Graph Fourier MMD (Exact)'
time_row = row

start_time = time.time()
G.compute_fourier_basis()
time_row['Compute Eigenvectors'] = time.time() - start_time

for i in range(5):
    start_time = time.time()
    row[f't=2^{4*i}'] = str(np.mean(GFMMD.locality(diffusions[:,:,i], method = 'exact')))
    runtime = time.time() - start_time
    time_row[f't=2^{4*i}'] = runtime

locality_measure.loc[1,:]=row
runtime_table.loc[1,:]=time_row

Using the Heat Filter with $h(\lambda) =  \lambda^{-1/2}\mathbf{1}\{\lambda \neq 0\}$, approximated by Chebyshev polynomials

In [184]:
G = Graph=pygsp.graphs.Minnesota()
GFMMD = Graph_Fourier_MMD(G)

row = {}
row['Method'] = 'Heat Filter (Chebyshev)'
time_row = row

time_row['Compute Eigenvectors'] = 0

for i in range(5):
    start_time = time.time()
    row[f't=2^{4*i}'] = str(np.mean(GFMMD.locality(diffusions[:,:,i], method = 'chebyshev', filter_name = 'heat')))
    runtime = time.time() - start_time
    time_row[f't=2^{4*i}'] = runtime

locality_measure.loc[2,:]=row
runtime_table.loc[2,:]=time_row

2022-08-24 12:46:07,443:[WARNING](pygsp.graphs.graph.lmax): The largest eigenvalue G.lmax is not available, we need to estimate it. Explicitly call G.estimate_lmax() or G.compute_fourier_basis() once beforehand to suppress the warning.


Using the Heat Filter with $h(\lambda) =  \lambda^{-1/2}\mathbf{1}\{\lambda \neq 0\}$, exactly:

In [185]:
G = Graph=pygsp.graphs.Minnesota()
GFMMD = Graph_Fourier_MMD(G)

row = {}
row['Method'] = 'Heat Filter (Exact)'
time_row = row

start_time = time.time()
G.compute_fourier_basis()
time_row['Compute Eigenvectors'] = time.time() - start_time

for i in range(5):
    start_time = time.time()
    row[f't=2^{4*i}'] = str(np.mean(GFMMD.locality(diffusions[:,:,i], method = 'exact', filter_name = 'heat')))
    runtime = time.time() - start_time
    time_row[f't=2^{4*i}'] = runtime

locality_measure.loc[3,:]=row
runtime_table.loc[3,:]=time_row

Using Diffusion EMD to embed the signals (such that the $L_1$ distance betwen embeddings is equal to the EMD using 
the modified diffusion distance $D_\alpha$) and then taking the $L_1$ distance to the embedding of the uniform distribution

In [188]:
row = {}
row['Method'] = 'Diffusion EMD'
time_row = row

time_row['Compute Eigenvectors'] = 0

adj = G.A
dc = DiffusionCheb()

for i in range(5):
    distributions = np.hstack((np.ones(G.N).reshape(G.N,1) / G.N, diffusions[:,:,i]))

    # Embeddings where the L1 distance approximates the Earth Mover's Distance
    start_time = time.time()
    embeddings = dc.fit_transform(adj, distributions)
    # Shape: (5, 60)

    dist_to_uniform = []
    for j in range(1,distributions.shape[1]):
        dist_to_uniform.append(np.linalg.norm(embeddings[j] - embeddings[0], ord=1))
    row[f't=2^{4*i}'] = np.mean(dist_to_uniform)

    runtime = time.time() - start_time
    time_row[f't=2^{4*i}'] = runtime
    
locality_measure.loc[4,:]=row
runtime_table.loc[4,:]=time_row


In [196]:
locality_measure.style.to_latex('Documents/Graph Fourier MMD/Latex/Distance.tex')
runtime_table.style.to_latex('Documents/Graph Fourier MMD/Latex/Time.tex')